In [5]:
import pandas as pd
from optimization2 import getServiceArea
import geopandas as gpd
from shapely.geometry import Point

OSError: File b'input_data/combined_buildings_2.csv' does not exist

# Input grid centroids

In [11]:
points_to_assess = pd.read_csv('input_data/points_to_assess_locations_3.csv')


# Select output path

In [12]:
path = 'location_results/results_point_location.csv'

# Run Model for grid locations

In [15]:
metric = 'energy'
direct = 0

In [16]:
X_lat_lon = list(zip(points_to_assess['y_lat'],points_to_assess['x_lon']))

In [17]:
for item in X_lat_lon:
    points = getServiceArea(item, path, metric, 7, -0.3, 0, 0, direct)

In [18]:
data = pd.read_csv(path, header=None)
data.columns= [['latlon', 'people', 'GHG', 'pumping', 'treatment', 'treatment_embodied', 'piping']]
data[['lat', 'lon']] = data['latlon'].str[1:-1].str.split(',', expand=True).astype(float)

# Convert data to Geodataframe

In [19]:
geometry = [Point(xy) for xy in zip(data.lon, data.lat)]
crs = {'init': 'epsg:4326'}
points_geo_df = gpd.GeoDataFrame(data, crs=crs, geometry=geometry)

In [21]:
points_geo_df.to_file('location_results/results_large_grid_energy_breakdown.shp')

# Input grid polygons

In [48]:
grid_large = gpd.read_file('input_data/grid_large_pop.shp')

# Clean up data to get rid of potential outliers and calculate mean size for each grid cell

In [49]:
points_grid = gpd.sjoin(points_geo_df, grid_large, how="left", op='intersects')

In [50]:
points_grid_sorted = points_grid.sort('people')

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [51]:
grid_large_2 = grid_large.copy()

In [52]:
def clean_mean(group, column_name, mean_column_name):
    if group[column_name].max()/group[column_name].min()>10:
        new_group = group[(group[column_name]<group[column_name].quantile(0.9)) & (group[column_name]>group[column_name].quantile(0.1))]
        return new_group[mean_column_name].mean()
        
    else:
        return group[mean_column_name].mean()     

In [53]:
points_grid_grouped = points_grid_sorted[['GHG', 'people', 'pumping', 'treatment', 'treatment_embodied', 'piping']].groupby(points_grid_sorted['index_right'])

In [54]:
mean_people_grid = {}
for i in range(max(points_grid_grouped.groups.keys())+1):
    group = points_grid_grouped.get_group(i)
    mean_people = clean_mean(group, 'people', 'people')
    mean_GHG = clean_mean(group, 'people', 'GHG')
    mean_pumping = clean_mean(group, 'people', 'pumping')
    mean_treatment = clean_mean(group, 'people', 'treatment')
    mean_treatment_embodied = clean_mean(group, 'people', 'treatment_embodied')
    mean_piping = clean_mean(group, 'people', 'piping')
    grid_large_2.set_value(i, 'people_mean', mean_people)
    grid_large_2.set_value(i, 'GHG_mean', mean_GHG)
    grid_large_2.set_value(i, 'pumping_mean', mean_pumping)
    grid_large_2.set_value(i, 'treatment_mean', mean_treatment)
    grid_large_2.set_value(i, 'treatment_embodied_mean', mean_treatment_embodied)
    grid_large_2.set_value(i, 'piping_mean', mean_piping)

In [55]:
grid_large_join = grid_large.join(grid_large_2, rsuffix='_right')
grid_large_join_2 = grid_large_join.drop(['Id_right', 'geometry_right', 'SUM_pop_right','SUM_pop_co_right','SUM_pop_re_right'], 1)

# Save to file

In [56]:
grid_large_join_2.to_file('location_results/grid_cell_output.shp')